<a href="https://colab.research.google.com/github/izzul112/Submission-Image_Classification/blob/master/Submission_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# melakukan ekstraksi pada file zip
import zipfile,os
local_zip = '/content/gdrive/My Drive/dataset/dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp'
train_dir = os.path.join(base_dir, 'dataset')

In [4]:
os.listdir('/tmp/dataset')

['CNV', 'DRUSEN', 'DME', 'NORMAL']

In [5]:
# membuat direktori CNV pada direktori data training
train_CNV_dir = os.path.join(train_dir, 'CNV')

# membuat direktori DME pada direktori data training
train_DME_dir = os.path.join(train_dir, 'DME')

# membuat direktori DRUSEN pada direktori data training
train_DRUSEN_dir = os.path.join(train_dir, 'DRUSEN')

# membuat direktori NORMAL pada direktori data training
train_NORMAL_dir = os.path.join(train_dir, 'NORMAL')

# membuat direktori CNV pada direktori data validasi
validation_CNV_dir = os.path.join(train_dir, 'CNV')

# membuat direktori DME pada direktori data validasi
validation_DME_dir = os.path.join(train_dir, 'DME')

# membuat direktori DRUSEN pada direktori data validasi
validation_DRUSEN_dir = os.path.join(train_dir, 'DRUSEN')

# membuat direktori NORMAL pada direktori data validasi
validation_NORMAL_dir = os.path.join(train_dir, 'NORMAL')

In [6]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    validation_split=0.2,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest'
                    )

val_datagen = ImageDataGenerator(
                    rescale=1./255,
                    validation_split=0.2)

In [7]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(200, 300),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena kita merupakan masalah klasifikasi 2 kelas lebih maka menggunakan class_mode = 'categorical'
        class_mode="categorical",
        subset='training'
        )

validation_generator = train_datagen.flow_from_directory(
        val_datagen, # direktori data validasi
        target_size=(200, 300), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena kita merupakan masalah klasifikasi 2 kelas lebih maka menggunakan class_mode = 'categorical'
        class_mode='categorical',
        subset='validation'
        )

Found 9600 images belonging to 4 classes.


TypeError: ignored

In [4]:
# Callback

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') & logs.get('val_accuracy')>0.95):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(200, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.RMSprop(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
            train_generator,
            steps_per_epoch=25,  # berapa batch yang akan dieksekusi pada setiap epoch
            epochs=20, 
            batch_size=32,
            validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
            validation_steps=5,  # berapa batch yang akan dieksekusi pada setiap epoch
            verbose=2, callbacks=[callbacks])

Epoch 1/20
25/25 - 9s - loss: 1.4181 - accuracy: 0.2800 - val_loss: 1.3737 - val_accuracy: 0.3500
Epoch 2/20
25/25 - 8s - loss: 1.3876 - accuracy: 0.2700 - val_loss: 1.3823 - val_accuracy: 0.3500
Epoch 3/20
25/25 - 8s - loss: 1.3875 - accuracy: 0.2800 - val_loss: 1.3961 - val_accuracy: 0.2500
Epoch 4/20
25/25 - 8s - loss: 1.3930 - accuracy: 0.2200 - val_loss: 1.3910 - val_accuracy: 0.2000
Epoch 5/20
25/25 - 8s - loss: 1.4354 - accuracy: 0.3100 - val_loss: 1.3907 - val_accuracy: 0.1000
Epoch 6/20
25/25 - 8s - loss: 1.3860 - accuracy: 0.3300 - val_loss: 1.3725 - val_accuracy: 0.3500
Epoch 7/20
25/25 - 8s - loss: 1.3902 - accuracy: 0.3000 - val_loss: 1.3862 - val_accuracy: 0.2000
Epoch 8/20
25/25 - 8s - loss: 1.3835 - accuracy: 0.3000 - val_loss: 1.3763 - val_accuracy: 0.3000
Epoch 9/20
25/25 - 8s - loss: 1.3951 - accuracy: 0.2600 - val_loss: 1.3804 - val_accuracy: 0.2500
Epoch 10/20
25/25 - 8s - loss: 1.3885 - accuracy: 0.2200 - val_loss: 1.3835 - val_accuracy: 0.2000
Epoch 11/20
25/25 -

KeyboardInterrupt: ignored

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# # Konversi model.
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()

In [ ]:
# with tf.io.gfile.GFile('model.tflite', 'wb') as f:
#   f.write(tflite_model)